# Izračun točk

In [2]:
import pandas as pd
import re

In [3]:
benchmarks = pd.read_csv("benchmarks.csv", names=["name", "sudoku", "result", "time"])

In [4]:
def kind(sudoku):
    match = re.search("(?P<kind>\w+)-\d+\.sdk", sudoku)
    if match:
        return match.group("kind")


benchmarks["kind"] = benchmarks.sudoku.apply(kind)

In [6]:
time_range = (
    benchmarks[benchmarks.result == "CORRECT"]
    .groupby("sudoku")
    .aggregate({"time": [("fastest", "min"), ("slowest", "max")]})
)
time_range.columns = time_range.columns.get_level_values(1)
benchmarks = pd.merge(benchmarks, time_range, on="sudoku", how="left")

In [7]:
incorrect = benchmarks.result != "CORRECT"
benchmarks.loc[incorrect, "time"] = benchmarks[incorrect].slowest * 2

benchmarks["relative_time"] = benchmarks.time / benchmarks.fastest

In [8]:
benchmarks

,name,sudoku,result,time,kind,fastest,slowest,relative_time
0,student-1,kletke-1.sdk,ERROR,NaN,kletke,NaN,NaN,NaN
1,student-1,kletke-10.sdk,ERROR,0.039622,kletke,0.003459,0.019811,11.454756
2,student-1,kletke-100.sdk,ERROR,0.002816,kletke,0.001408,0.001408,2.000000
3,student-1,kletke-11.sdk,ERROR,0.007602,kletke,0.003801,0.003801,2.000000
4,student-1,kletke-12.sdk,ERROR,NaN,kletke,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
9862,student-9,termometri-95.sdk,WRONG,0.434490,termometri,0.002630,0.217245,165.205323
9863,student-9,termometri-96.sdk,TIMEOUT,0.003302,termometri,0.001651,0.001651,2.000000
9864,student-9,termometri-97.sdk,WRONG,0.007382,termometri,0.003691,0.003691,2.000000
9865,student-9,termometri-98.sdk,WRONG,0.005312,termometri,0.002656,0.002656,2.000000


In [10]:
benchmarks.loc[benchmarks["sudoku"]=="obicajni-1.sdk"]

,name,sudoku,result,time,kind,fastest,slowest,relative_time
300,student-10,obicajni-1.sdk,CORRECT,0.000060,obicajni,0.000027,0.001594,2.222222
750,student-11,obicajni-1.sdk,CORRECT,0.000046,obicajni,0.000027,0.001594,1.703704
1200,student-12,obicajni-1.sdk,CORRECT,0.000228,obicajni,0.000027,0.001594,8.444444
1650,student-13,obicajni-1.sdk,CORRECT,0.000786,obicajni,0.000027,0.001594,29.111111
2100,student-14,obicajni-1.sdk,CORRECT,0.000040,obicajni,0.000027,0.001594,1.481481
2550,student-15,obicajni-1.sdk,CORRECT,0.000606,obicajni,0.000027,0.001594,22.444444
3000,student-16,obicajni-1.sdk,TIMEOUT,0.003188,obicajni,0.000027,0.001594,118.074074
3452,student-18,obicajni-1.sdk,CORRECT,0.000074,obicajni,0.000027,0.001594,2.740741
3904,student-2,obicajni-1.sdk,CORRECT,0.000064,obicajni,0.000027,0.001594,2.370370
4451,student-21,obicajni-1.sdk,CORRECT,0.000052,obicajni,0.000027,0.001594,1.925926


## Točke za pravilnost

- 5 točk - 100% pravilno rešenih običajnih sudokujev
- 4 točke - ≥50% pravilno rešenih običajnih sudokujev
- 3 točke - >0% pravilno rešenih običajnih sudokujev
- 0-2 točki - odvisno od pravilnosti pomožnih funkcij - se določi ročno

In [ ]:
number_of_correct = (
    benchmarks[(benchmarks.kind == "obicajni") & (benchmarks.result == "CORRECT")]
    .groupby("name")
    .size()
)

In [ ]:
number_of_correct.sort_values(ascending=False).plot.bar()

In [ ]:
total = len(set(benchmarks[benchmarks.kind == "obicajni"].sudoku))


def tocke_za_pravilnost(correct):
    if correct == total:
        return 5
    elif correct / total >= 0.5:
        return 4
    else:
        # če se vrednost pojavi, je moral biti vsaj en pravilen
        return 3

In [ ]:
number_of_correct.apply(tocke_za_pravilnost)

## Točke za hitrost

In [ ]:
speed_benchmarks = benchmarks[
    (benchmarks.kind == "obicajni") & ~pd.isna(benchmarks.time)
]

In [ ]:
speed_benchmarks.boxplot("relative_time", by="name", vert=False)

In [ ]:
speed = speed_benchmarks.groupby("name").mean().relative_time
speed.sort_values().plot.bar(log=True)
speed

## Točke za razširitve



In [ ]:
extension_benchmarks = benchmarks[
    (benchmarks.result != "ERROR") & (benchmarks.kind != "obicajni")
]

In [ ]:
extension_benchmarks.groupby(["name", "kind", "result"]).size().unstack()

In [ ]:
extension_benchmarks.groupby(["name", "kind"]).mean().relative_time